In [ ]:
#! /usr/bin/env python3

import numpy as np
import os
import argparse
from pypcd import pypcd
import csv
from tqdm import tqdm
import cv2
import natsort
import shutil


def convert_pcd_to_bin(args):
    ## Find all pcd files
    pcd_files = []
    for (path, dir, files) in os.walk(args.sync_pcd_path):
        for filename in files:
            # print(filename)
            ext = os.path.splitext(filename)[-1]
            if ext == ".pcd":
                pcd_files.append(path + "/" + filename)

    ## Sort pcd files by file name
    pcd_files.sort()
    print("Finish to load point clouds!")

    ## Make bin_path directory
    try:
        if not (os.path.isdir(args.sync_bin_path)):
            os.makedirs(os.path.join(args.sync_bin_path))
    except OSError as e:
        if e.errno != errno.EEXIST:
            print("Failed to create directory!!!!!")
            raise

    ## Generate csv meta file
    csv_file_path = os.path.join(args.sync_bin_path, "meta.csv")
    csv_file = open(csv_file_path, "w")
    meta_file = csv.writer(
        csv_file, delimiter=",", quotechar="|", quoting=csv.QUOTE_MINIMAL
    )
    ## Write csv meta file header
    meta_file.writerow(["pcd file name", "bin file name"])
    print("Finish to generate csv meta file")

    ## Converting Process
    print("Converting Start!")
    seq = 0

    for pcd_file in tqdm(pcd_files):
        ## Get pcd file
        pc = pypcd.PointCloud.from_path(pcd_file)

        ## Generate bin file name
        bin_file_name = "{:010d}.bin".format(seq)
        bin_file_path = os.path.join(args.sync_bin_path, bin_file_name)
        print("bin_file_name: {}".format(bin_file_name))

        ## Get data from pcd (x, y, z, intensity, ring, time)
        np_x = (np.array(pc.pc_data["x"], dtype=np.float32)).astype(np.float32)
        np_y = (np.array(pc.pc_data["y"], dtype=np.float32)).astype(np.float32)
        np_z = (np.array(pc.pc_data["z"], dtype=np.float32)).astype(np.float32)

        np_i = (np.array(pc.pc_data[args.intensity], dtype=np.float32)).astype(
            np.float32
        )

        ## Stack all data
        points_32 = np.transpose(np.vstack((np_x, np_y, np_z, np_i)))
        seq = seq + 1
        points_32.tofile(bin_file_path)


def sync_data(args):
    if not os.path.exists(args.sync_images_path):
        os.makedirs(args.sync_images_path)
    if not os.path.exists(args.sync_pcd_path):
        os.makedirs(args.sync_pcd_path)
    if not os.path.exists(args.sync_calib_path):
        os.makedirs(args.sync_calib_path)
    if not os.path.exists(args.sync_label_path):
        os.makedirs(args.sync_label_path)

    images_file_list = os.listdir(args.images_path)
    images_file_list = natsort.natsorted(images_file_list)

    pcds_file_list = os.listdir(args.pcd_path)
    pcds_file_list = natsort.natsorted(pcds_file_list)

    calib_file_list = os.listdir(args.calib_path)
    calib_file_list = natsort.natsorted(calib_file_list)

    label_file_list = os.listdir(args.label_path)
    label_file_list = natsort.natsorted(label_file_list)

    pivot_file_list = []
    compared_file_list = []

    if len(images_file_list) <= len(pcds_file_list):
        pivot_file_list = images_file_list
        compared_file_list = pcds_file_list
        pivot_path = args.images_path
        cp_path = args.pcd_path
        sync_pivot_path = args.sync_images_path
        sync_cp_path = args.sync_pcd_path
        pivot_file_type = ".png"
        cp_file_type = ".pcd"
    else:
        pivot_file_list = pcds_file_list
        compared_file_list = images_file_list
        pivot_path = args.pcd_path
        cp_path = args.images_path
        sync_pivot_path = args.sync_pcd_path
        sync_cp_path = args.sync_images_path
        pivot_file_type = ".pcd"
        cp_file_type = ".png"

        # pivot_index = min(len(images_file_list),len(pcds_file_list))
        # pivot_file_list = images_file_list

        # pivot_file_list = pcds_file_list if
    #  +str(idx).zfill(10)+".png"
    idx = 0
    for _, pivot_file in enumerate(pivot_file_list):

        try:
            index = compared_file_list.index(pivot_file[:-4] + cp_file_type)
            # print(index)
            pivot_src = os.path.join(pivot_path, pivot_file)
            cp_src = os.path.join(cp_path, compared_file_list[index])
            calib_src = os.path.join(args.calib_path, calib_file_list[idx])
            label_src = os.path.join(args.label_path, label_file_list[idx])

            shutil.copy2(
                pivot_src, sync_pivot_path + str(idx).zfill(10) + pivot_file_type
            )

            shutil.copy2(cp_src, sync_cp_path + str(idx).zfill(10) + cp_file_type)
            shutil.copy2(calib_src, args.sync_calib_path + str(idx).zfill(10) + ".txt")
            shutil.copy2(label_src, args.sync_label_path + str(idx).zfill(10) + ".txt")
            idx += 1

        except:
            print(f"can not find")


def main():
    ## Add parser
    parser = argparse.ArgumentParser(description="Convert .pcd to .bin and sync data")
    parser.add_argument(
        "--pcd_path", help=".pcd file path.", type=str, default="./_kitti/raw_data_pcd/"
    )
    parser.add_argument(
        "--sync_pcd_path",
        help=".bin file path.",
        type=str,
        default="./_kitti/raw_data_pcd/bin/",
    )
    parser.add_argument(
        "--calib-path", help="calib file path.", type=str, default="./_kitti/raw_calib/"
    )
    parser.add_argument(
        "--label-path", help="label file path.", type=str, default="./_kitti/raw_label/"
    )
    parser.add_argument("--file_name", help="File name.", type=str, default="file_name")
    parser.add_argument("--intensity", help="intensity or rgb", type=str, default="rgb")
    parser.add_argument(
        "--images-path", help="images file path.", type=str, default="_kitti/raw_image/"
    )
    parser.add_argument(
        "--sync-images-path",
        help="synced images file path.",
        type=str,
        default="_kitti/training/image_00/",
    )
    parser.add_argument(
        "--sync-bin-path",
        help="synced bin file path.",
        type=str,
        default="_kitti/training/velodyne_points/",
    )
    parser.add_argument(
        "--sync-calib-path",
        help="synced calib file path.",
        type=str,
        default="_kitti/training/calib/",
    )
    parser.add_argument(
        "--sync-label-path",
        help="synced label file path.",
        type=str,
        default="_kitti/training/label_2/",
    )
    # oxts_path = "_kitti/oxts/data/"

    args = parser.parse_args()
    sync_data(args)
    convert_pcd_to_bin(args)


if __name__ == "__main__":
    main()